In [5]:
#only for testing
def return_question():
    import random
    next_ques=random.choice(["Yooooo!",'Heyyyy!','Weeeee!'])
    pic_link='https://bakingamoment.com/wp-content/uploads/2018/02/IMG_6207-chocolate-molten-lava-cakes-recipe-square.jpg'
    return next_ques,pic_link


from tkinter import *
from tkinter import scrolledtext
import random
from random import shuffle

#functions to be implemented: connect each page to the randomly generated question
#user can choose from a multiple choice question
#user is told right or wrong
#count the correct answer number
#quit button
class quizGUI:
    
    database = []
    
    def __init__(self):
        root = Tk()
        root.lift()
        root.attributes("-topmost", True)
        root.geometry("800x600")
        root.title("MET-Q")
        self.root = root
        
        # Build GUI
        canvas_width = 800
        canvas_height = 600
        canvas = Canvas(root, height=canvas_height, width=canvas_width)
        
        #correct/incorrect indicator
        var=StringVar()
        self.cor_label=Label(root,textvariable=var)
        self.var=var
        self.cor_label.pack()
        #title label
        self.title_label = Label(root, text="The MET Quiz", font = "Times 20 bold").pack(side="top")
    
        #record the number of correct answers so far
        self.total = IntVar() #total correct answers
        self.total_var=StringVar()
        self.total_var.set('Number of correct answers: 0')
        self.score_label = Label(root,textvariable=self.total_var)
        self.score_label.pack()
        
        #record the number of questions completed so far
        self.num_var=StringVar()
        self.num=IntVar()
        self.num_var.set('Questions completed: 0')
        self.num_of_ques=Label(root,textvariable=self.num_var)
        self.num_of_ques.pack()
    
        #question stem
        obj_title = StringVar()  # object string
        question_var=StringVar() # question string
        self.Q = Label(root, textvariable=question_var).pack() #question label
        question_var.set(return_question())
        self.question_var=question_var
        self.obj_label = Label(root, textvariable=obj_title).pack() #add object title
        
        
        
        #Question picture
        pic=PhotoImage()
        
        
        
        
        #initialize choice buttons
        self.A = Button(root, text="A",command=lambda: self.correct())
        self.A.pack(padx=5,pady=5)
        self.B = Button(root, text="B",command=lambda: self.incorrect())
        self.B.pack(padx=5,pady=5)
        self.C = Button(root, text="C",command=lambda: self.incorrect())
        self.C.pack(padx=5,pady=5)
        self.D = Button(root, text="D",command=lambda: self.incorrect())
        self.D.pack(padx=5,pady=5)
        self.next=Button(root, text="Next question",command=lambda:self.next_()).pack()

        #quit button
        self.close_button=Button(root, text="Quit", command=lambda: root.destroy())
        self.close_button.pack(padx=20,side='right')
        
        frame=Frame(root)
        frame.pack()
        canvas.pack()
        
        root.mainloop() 
        

    #add 1 to score if the correct answer is chosen
    def correct(self):
        self.var.set('Correct')
        self.total.set(self.total.get() + 1) 
        self.total_var.set('Number of correct answers: '+str(self.total.get()))
    
    #display text when incorrect answer is chosen
    def incorrect(self):
        self.var.set('Incorrect')
        
    #generate another question and set up random question and choices
    #refer to create questions file
#     def display_next_question(self, self.total, self.question, self.obj_title, self.A, self.B, self.C, self.D)
#         pass
        
    def next_(self):
        self.question_var.set(return_question()[0])
        self.num.set(self.num.get()+1)
        self.num_var.set('Questions completed: '+str(self.num.get()))

       

In [6]:
myGUI = quizGUI()